# Python LEGB scopes

See https://realpython.com/python-scope-legb-rule/

In programming, the scope of a name defines the area of a program in which you can unambiguously access that name, 
such as variables, functions, objects, and so on. A name will only be visible to and accessible by the code in its 
scope. Several programming languages take advantage of scope for avoiding name collisions and unpredictable 
behaviors. Most commonly, you’ll distinguish two general scopes:
    
1. Global scope: The names that you define in this scope are available to all your code.
1. Local scope: The names that you define in this scope are only available or visible to the code within the scope.

Names and Scopes in Python
Since Python is a dynamically-typed language, variables in Python come into existence when you first assign them a value. On the other hand, functions and classes are available after you define them using def or class, respectively. Finally, modules exist after you import them. As a summary, you can create Python names through one of the following operations:

| Operation | Statement |
|:--------- |:--------- |
| Assignments | `x = value` |
| Import operations | `import module or from module import name` |
| Function definitions | `def my_func(): ...` |
| Argument definitions in the context of functions | `def my_func(arg1, arg2,... argN): ...` |
| Class definitions	| `class MyClass: ...` |

All these operations create or, in the case of assignments, update new Python names because all of them assign a name to a variable, constant, function, class, instance, module, or other Python object.

Python scopes are implemented as dictionaries that map names to objects. These dictionaries are commonly called namespaces. These are the concrete mechanisms that Python uses to store names. They’re stored in a special attribute called .__dict__.

Names at the top level of a module are stored in the module’s namespace. In other words, they’re stored in the module’s .__dict__ attribute.

In [185]:
import sys
sys.__dict__.keys()

dict_keys(['__name__', '__doc__', '__package__', '__loader__', '__spec__', 'addaudithook', 'audit', 'breakpointhook', 'callstats', '_clear_type_cache', '_current_frames', 'displayhook', 'exc_info', 'excepthook', 'exit', 'getdefaultencoding', 'getdlopenflags', 'getallocatedblocks', 'getfilesystemencoding', 'getfilesystemencodeerrors', 'getrefcount', 'getrecursionlimit', 'getsizeof', '_getframe', 'intern', 'is_finalizing', 'setcheckinterval', 'getcheckinterval', 'setswitchinterval', 'getswitchinterval', 'setdlopenflags', 'setprofile', 'getprofile', 'setrecursionlimit', 'settrace', 'gettrace', 'call_tracing', '_debugmallocstats', 'set_coroutine_origin_tracking_depth', 'get_coroutine_origin_tracking_depth', 'set_asyncgen_hooks', 'get_asyncgen_hooks', 'unraisablehook', 'modules', 'stderr', '__stderr__', '__displayhook__', '__excepthook__', '__breakpointhook__', '__unraisablehook__', 'version', 'hexversion', '_git', '_framework', 'api_version', 'copyright', 'platform', 'maxsize', 'float_info

In [190]:
if sys.__name__ == "__main__":
    print("bla")

Accessing scope

In [4]:
sys.__dict__['__name__']

'sys'

Python resolves names using the so-called **LEGB** rule, which is named after the Python scope for names. The letters in LEGB stand for **Local**, **Enclosing**, **Global**, and **Built-in**. Here’s a quick overview of what these terms mean:

## Local (or function) scope

Local (or function) scope is the code block or body of any Python function or lambda expression. This Python scope contains the names that you define inside the function. These names will only be visible from the code of the function. It’s created at **function call**, not at **function definition**, so you’ll have as many different local scopes as function calls. This is true even if you call the same function multiple times, or recursively. Each call will result in a new local scope being created.

In [198]:
def square(base = False):
    result = base ** 2
    print(f'The square of {base} is: {result}')
    return True

square(10)
square(4)

The square of 10 is: 100
The square of 4 is: 16


True

In [193]:
square(10)
result  # Isn't accessible from outside square()
base  # Isn't accessible from outside square()

The square of 10 is: 100


NameError: name 'result' is not defined

In [196]:
square.__code__.co_varnames # variable names

('base', 'result')

In [108]:
square.__code__.co_argcount # amount of aruments it takes

1

In [199]:
square.__code__.co_consts #constants in function

(None, 2, 'The square of ', ' is: ', True)

In [105]:
square.__code__.co_name # function name

'square'

How many local scopes?

for base?   
for x in a?   
for x in b?   

In [202]:
timesCalled = 0
def sq(x):
    global timesCalled
    timesCalled += 1
    return x ** 2

sq(4)
print(timesCalled)
        
a = list(sq(x) for x in range(10, 15))
b = [(lambda x: x*x)(x) for x in range (-2, 17)]

1


In [53]:
print("scope x in sq: {}".format(timesCalled)) 
print("scope x in b: {}".format(2 * len(b)))
print("scope x in a: {}".format(len(a)))


scope x in sq: 5
scope x in b: 38
scope x in a: 5


In [79]:
from math import sqrt

def rsq(x, calls = 0):
    y = (x ** sqrt(x)) / sqrt(x)
      
    if y < (10 ** 10):
        rsq(y, calls + 1)
    else:
        
        print("took {} iterations to reach {}".format(calls, y))
        
rsq(4)
rsq(3)


took 2 iterations to reach 4.147577427414219e+22
took 3 iterations to reach 8302939159482839.0


You can inspect the names and parameters of a function using .__code__, which is an attribute that holds information on the function’s internal code. 

## Enclosing (or nonlocal) scope

Enclosing (or nonlocal) scope is a special scope that only exists for nested functions. If the local scope is an inner or nested function, then the enclosing scope is the scope of the outer or enclosing function. This scope contains the names that you define in the enclosing function. The names in the enclosing scope are visible from the code of the inner and enclosing functions.

In [216]:
def outer_func():
    # This block is the Local scope of outer_func()
    var1 = 100  # A nonlocal var
        
    # It's also the enclosing scope of inner_func()
    def inner_func():
        # This block is the Local scope of inner_func()
        print(f"Printing var from inner_func(): {var1}")
        #print(f"Printing another_var from inner_func(): {another_var}")
        
        def inner_inner_func():
            nonlocal var1
            var1 +=1
            print(f"Printing var from inner_inner_func(): {var1}")
            
        inner_inner_func()
        
    inner_func()
    #another_var = 200  # This is defined after calling inner_func()
    print(f"Printing var from outer_func(): {var1}")

print("outer func")
outer_func()

#print("inner func, should error")
#inner_func()

outer func
Printing var from inner_func(): 100
Printing var from inner_inner_func(): 101
Printing var from outer_func(): 101


## Global (or module) scope


Global (or module) scope is the top-most scope in a Python program, script, or module. This Python scope contains all of the names that you define at the top level of a program or a module. Names in this Python scope are visible from everywhere in your code.

In [223]:
len(dir())  == len(__main__.__dict__)


True

In [149]:
import __main__

var = 10000

print(__main__.__dict__['var'])

## Use dir to get the global scope
dir()[-1] # get the last one



10000


'var'

In [208]:
var = 10000

def func():
    global var
    var = 200 # All of a sudden var in local
    print("Var from func: {}".format(var))
    
func()
print("Var from global: {}".format(var))

Var from func: 200
Var from global: 200


Another example with scopes

In [134]:
from random import random

g = random()
class MyClass:
    def x():
        pass
    
class MyOtherClass(MyClass):
    global g
    h = 12
    def y():
        G = 100
        def yy():
            nonlocal G
            G += 1
            return g 
        
        print("Random number from MyOtherClass.y.yy: {}".format(yy()))
        return G
    
    
print(MyClass.__dict__)
print(MyOtherClass.__dict__)

print(MyOtherClass.y())
#print(__main__.__dict__)

{'__module__': '__main__', 'x': <function MyClass.x at 0x7f2a20084550>, '__dict__': <attribute '__dict__' of 'MyClass' objects>, '__weakref__': <attribute '__weakref__' of 'MyClass' objects>, '__doc__': None}
{'__module__': '__main__', 'h': 12, 'y': <function MyOtherClass.y at 0x7f2a0bfbdc10>, '__doc__': None}
Random number from MyOtherClass.y.yy: 0.6903161716600756
101


## LEG scopes

In [161]:
# This area is the global or module scope
number = 100
def outer_func():
    var = 10
    # This block is the local scope of outer_func()
    # It's also the enclosing scope of inner_func()
    def inner_func():
        # This block is the local scope of inner_func()
        print(number)

    inner_func()
        
outer_func()

100


## Built-in scope 

Built-in scope is a special Python scope that’s created or loaded whenever you run a script or open an interactive session. This scope contains names such as keywords, functions, exceptions, and other attributes that are built into Python. Names in this Python scope are also available from everywhere in your code. It’s automatically loaded by Python when you run a program or script.

In [229]:
abs(-13)


13

In [162]:

dir(__builtins__)

['ArithmeticError',
 'AssertionError',
 'AttributeError',
 'BaseException',
 'BlockingIOError',
 'BrokenPipeError',
 'BufferError',
 'BytesWarning',
 'ChildProcessError',
 'ConnectionAbortedError',
 'ConnectionError',
 'ConnectionRefusedError',
 'ConnectionResetError',
 'DeprecationWarning',
 'EOFError',
 'Ellipsis',
 'EnvironmentError',
 'Exception',
 'False',
 'FileExistsError',
 'FileNotFoundError',
 'FloatingPointError',
 'FutureWarning',
 'GeneratorExit',
 'IOError',
 'ImportError',
 'ImportWarning',
 'IndentationError',
 'IndexError',
 'InterruptedError',
 'IsADirectoryError',
 'KeyError',
 'KeyboardInterrupt',
 'LookupError',
 'MemoryError',
 'ModuleNotFoundError',
 'NameError',
 'None',
 'NotADirectoryError',
 'NotImplemented',
 'NotImplementedError',
 'OSError',
 'OverflowError',
 'PendingDeprecationWarning',
 'PermissionError',
 'ProcessLookupError',
 'RecursionError',
 'ReferenceError',
 'ResourceWarning',
 'RuntimeError',
 'RuntimeWarning',
 'StopAsyncIteration',
 'StopIter

In [228]:
__main__.__dict__.keys()

dict_keys(['__name__', '__doc__', '__package__', '__loader__', '__spec__', '__builtin__', '__builtins__', '_ih', '_oh', '_dh', 'In', 'Out', 'get_ipython', 'exit', 'quit', '_', '__', '___', '_i', '_ii', '_iii', '_i1', 'sys', '_1', '_i2', '_2', '_i3', '_3', '_i4', '_4', '_i5', '_i6', 'square', '_i7', '_i8', '_i9', '_i10', '_i11', '_11', '_i12', '_i13', '_i14', '_14', '_i15', '_15', '_i16', '_i17', '_17', '_i18', '_18', '_i19', 'sq', '_19', '_i20', '_20', '_i21', '_21', '_i22', '_22', '_i23', '_23', '_i24', 'a', 'b', '_i25', '_25', '_i26', '_26', '_i27', '_i28', '_28', '_i29', '_i30', '_i31', 'rsq', '_i32', '_i33', '_i34', '_i35', '_i36', '_i37', '_i38', '_i39', '_i40', '_i41', '_41', '_i42', '_i43', '_i44', '_i45', '_i46', '_i47', '_i48', '_i49', 'timesCalled', '_i50', '_i51', '_i52', '_i53', '_i54', '_i55', 'sqrt', '_i56', '_i57', '_i58', '_i59', '_i60', '_i61', '_i62', '_i63', '_i64', '_i65', '_i66', '_i67', '_i68', '_i69', '_i70', '_i71', '_i72', '_i73', '_i74', '_i75', '_i76', '_i77'

In [232]:
import builtins as jemoeder
from random import randint

jemoeder is __builtins__

print(sum(2, 3))

def (x, y):
    return x-y 

print(jemoeder.sorted([randint(-x, x) for x in range(20)]))
print(__builtins__.sum(range(100)))
print(sum(2, 3))

-1
[-18, -16, -9, -6, -5, -5, -4, -4, -1, 0, 1, 1, 1, 1, 3, 5, 5, 10, 12, 16]
4950
-1


In [238]:
import builtins

builtins.mysq = (lambda x: x*x)
builtins.var2 = 9



print(__builtins__.mysq(4))
print(builtins.mysq(__builtins__.var))

16
81


In [239]:
mysq(4)
var2



9

## What doe StackOverflow say???!!1

https://stackoverflow.com/questions/7054228/accessing-a-function-within-a-functionnested-function

Actually, a concise rule for Python Scope resolution, from [Learning Python, 3rd. Ed.](https://www.amazon.com/dp/0596513984). (These rules are specific to variable names, not attributes. If you reference it without a period, these rules apply.)

LEGB Rule

Local — Names assigned in any way within a function (`def` or `lambda`), and not declared global in that function

Enclosing-function — Names assigned in the local scope of any and all statically enclosing functions (`def` or `lambda`), from inner to outer

Global (module) — Names assigned at the top-level of a module file, or by executing a `global` statement in a `def` within the file

Built-in (Python) — Names preassigned in the built-in names module: `open`, `range`, `SyntaxError`, etc

So, in the case of

```python

code1 # global
class Foo:
    code2 # local 
    def spam(): # enclosed
        global code1 # global in enclosed place
        code3 # local
        for code4: # local
            code5 #local
            x()
```

The for loop does not have its own namespace. In LEGB order, the scopes would be

* `L`: Local in `def spam` (in `code3`, `code4`, and `code5`)      
* `E`: Any enclosing functions (if the whole example were in another `def`)     
* `G`: Were there any `x` declared globally in the module (in `code1`)?     
* `B`: Any builtin `x` in Python.    


`x` will never be found in `code2` (even in cases where you might expect it would, see [Antti's answer](https://stackoverflow.com/questions/291978/short-description-of-the-scoping-rules/23471004#23471004) or [here](https://stackoverflow.com/questions/13905741/accessing-class-variables-from-a-list-comprehension-in-the-class-definition)).

